In [1]:
%cd ../

D:\Projects\EpiTB\tbdx23


In [3]:
import os
import pandas as pd
import numpy as np
from model.core.inputs import load_inputs
from model.ind.dy import ModelIND
from scipy.optimize import minimize_scalar
from scipy.interpolate import interp1d

In [4]:
iso = 'IND'
in_folder = f'out/A_{iso}'
out_folder = f'out/B_{iso}'

In [5]:
os.makedirs(out_folder, exist_ok=True)

In [7]:
inp = load_inputs(f'pars/{iso}')
model = ModelIND(inp)

In [8]:
tarq = pd.read_csv(f'pars/{iso}/targets_q.csv').set_index('Time')
tarq

,iso,Country,Year,q,QCNR
Time,,,,,
2020.125,IND,India,2020,1,0.000378
2020.375,IND,India,2020,2,0.000236
2020.625,IND,India,2020,3,0.000260
2020.875,IND,India,2020,4,0.000295
2021.125,IND,India,2021,1,0.000372
2021.375,IND,India,2021,2,0.000262
2021.625,IND,India,2021,3,0.000398
2021.875,IND,India,2021,4,0.000350
2022.125,IND,India,2022,1,0.000379


In [9]:
particles = pd.read_csv(f'{in_folder}/Post.csv')
particles = [dict(row) for _, row in particles.iterrows()]

sims = list()
for p in particles:
    ys, _, _ = model.simulate_to_preCOVID(p)
    sims.append((p, ys))

In [10]:
def fn(k, sims, tarq, t0, i0, t_in, y_in0):
    y_in0 = y_in0.copy()
    y_in0[i0] = k
    k_covid = interp1d(t_in, y_in0, kind='nearest', bounds_error=False, fill_value=1)

    ds = list()

    for p, ys in sims:
        p['k_covid'] = k_covid
        _, ms, _ = model.simulate_to_postCOVID(p, ys, t_eval=np.array([t0 - 1/8, t0 + 1/8]))
        d = (tarq.QCNR[t0] / ms.CNR - 1).pow(2).sum()
        ds.append(d)

    return sum(ds)

In [11]:
ts = list(tarq.index)
p, ys = sims[1]

t_in = np.array([2019.875] + ts)
y_in = np.ones_like(t_in)

sims0 = sims[:50]

for i, t in enumerate(t_in[2:], 2):
#     if t > 2022:
#         continue
    
    opt = minimize_scalar(fn, args=(sims0, tarq, t, i, t_in, y_in, ), method='bounded', bounds=[0.2, 1.5])
    
    # Update k_covid
    y_in[i] = opt.x
    k_covid = interp1d(t_in, y_in, kind='nearest', bounds_error=False, fill_value=1)

    sims1 = list()
    print(t, i, opt.x)
    for p, ys in sims0:
        p['k_covid'] = k_covid
        ys1, _, _ = model.simulate_to_postCOVID(p, ys, t_eval=np.array([t - 1/8, t + 1/8]))
        sims1.append((p, ys1))

    sims0 = sims1


2020.375 2 1.499996589196675


KeyboardInterrupt: 

In [10]:
ts = list(tarq.index)
ts += [ts[-1] + 1 / 4]
ts = np.array(ts)

k_covid = interp1d(t_in, y_in, kind='nearest', bounds_error=False, fill_value=1)

mss = list()

for i, (p, ys) in enumerate(sims):
    p['k_covid'] = k_covid
    _, ms, _ = model.simulate_to_postCOVID(p, ys, t_eval=ts)
    mss.append(ms.assign(Key = i))

mss = pd.concat(mss)

In [11]:
pd.DataFrame({'Time': t_in, 'k_covid': y_in}).to_csv(f'{out_folder}/k_covid.csv')

In [12]:
mss.to_csv(f'{out_folder}/RunPost.csv')

In [13]:
pss = pd.DataFrame([p for p, _ in sims])
pss.to_csv(f'{out_folder}/Post.csv')